In [ ]:
import os
import time
import shutil
import PIL
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D
Activation,Dropout,BatchNormalization, Input,  Concatenate,
GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D

from tensorflow.keras import Model,regularizers,Sequential
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, Adamax
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# from tensorflow.keras.applications.efficientnet

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
from IPython.core.display import display, HTML

In [ ]:
df = df.sample(12)
plt.figure(figsize=(20,20))
for i in range(len(df)):
    #print(df.iloc[i]['filepaths'])
    path = '../input/100-bird-species/' + df.iloc[i]['filepaths']
    label = df.iloc[i]['labels']
    Type = df.iloc[i]['data set']
    plt.subplot(3, 4, i+1)
    img = tf.keras.preprocessing.image.load_img(path)
    plt.imshow(img)
    plt.xlabel(f'{label} : {Type}')
plt.show()

In [ ]:
class_names=train_ds.class_names
class_count=len(class_names)
plt.figure(figsize=(20,20))
for images, labels in train_ds.take(1):
    for i in range (25):
        plt.subplot(5,5,i +1)
        img=images[i]/255         
        plt.title(class_names[labels[i]], color='blue', fontsize=12)
        plt.imshow(img)
        plt.axis('off')
    plt.show()

In [ ]:
train_data = ImageDataGenerator(preprocessing_function = preprocess_input,
                                rotation_range=20, horizontal_flip=True)

train_generator = train_data.flow_from_directory('train',
                                                batch_size=64,
                                                target_size=(224,224),
                                                class_mode='categorical')

In [ ]:
valid_data = ImageDataGenerator(preprocessing_function = preprocess_input,
                                rotation_range=20, horizontal_flip=True)

validation_generator = valid_data.flow_from_directory('valid',
                                                batch_size=64,
                                                target_size=(224,224),
                                                class_mode='categorical')

In [ ]:
for _ in range(5):
    img, label = train_generator.next()
    print(img.shape) 
    plt.imshow(img[0])
    plt.show()

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
#eff_model.summary()
vgg_model.trainable = False

layer0 = Flatten(name='flatten')(vgg_model.output)
layer1 = Dense(4096, activation='relu',name='fc1')(layer0)
layer2 = Dense(4096, activation='relu',name='fc2')(layer1)
out_layer = Dense(300, activation='softmax')(layer2)
vgg_model = Model(vgg_model.input, out_layer)
vgg_model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
vgg_model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
              #,
              #ModelCheckpoint(filepath='Fullmodel.{epoch:02d}-{acc:.2f}.h5', monitor='val_acc', save_best_only=False, mode='max')]
             ]

In [ ]:
history = vgg_model.fit(
      train_generator, 
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
test = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test.flow_from_directory(
        '../input/100-bird-species/test',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical')

In [ ]:
vgg_model.evaluate(test_generator,use_multiprocessing=True,workers=10)

# -------------------------------------------

In [ ]:
base_model=tf.keras.applications.EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max') 
x=base_model.output
base_model.trainable=False # Set base model as NOT trainable initially
x=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x=Dropout(rate=.45, seed=123)(x)        
output=Dense(class_count, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)

# Freezing pretrained layers
for layer in base_model.layers:
    layer.trainable=False
    
model.compile(Adamax(lr=.001), loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
epochs=10
ask_epoch=2
rlronp=tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5,  patience=1, verbose=1)
# callbacks=[rlronp,TRC(model=model, base_model=base_model,epochs=epochs,  ask_epoch=ask_epoch)]
history=model.fit( train_ds, validation_data=valid_ds, epochs=epochs, verbose=1, callbacks=rlronp)